In [1]:
import tensorflow as tf
import numpy as np

c:\users\muhammed buyukkinaci\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#hyperparameter'leri tanımlıyoruz.
#time-step gibi.
max_len = 20
step = 1
num_units = 256
learning_rate = 0.001
batch_size = 128
epoch = 50
temperature = 0.5

In [3]:
def read_data(file_name):
    '''
     open and read text file
    '''
    text = open(file_name, 'r',encoding="utf8").read()
    return text.lower()


In [4]:
def making_one_hot(text):
    '''
     one_hot hale getiriyoruz.
    '''
    #unique character sayısını buluyoruz.
    unique_chars = list(set(text))
    len_unique_chars = len(unique_chars)

    input_chars = []
    output_char = []
    #ilk 20 character(0:20) input, 21. character(20) output.  veya da (10500:10520) input, 10520 output.
    for i in range(0, len(text) - max_len, step):
        input_chars.append(text[i:i+max_len])
        output_char.append(text[i+max_len])
    #shape'i (len(input_chars), max_len, len_unique_chars) olan ve 0'lardan oluşan 3 boyutlu bir matrix oluşturuyoruz.
    train_data = np.zeros((len(input_chars), max_len, len_unique_chars))
    #shape'i (len(input_chars), len_unique_chars) olan ve 0'lardan oluşan 2 boyutlu bir matrix oluşturuyoruz
    target_data = np.zeros((len(input_chars), len_unique_chars))
    #one-hot formatına çeviriyoruz.
    for i , each in enumerate(input_chars):
        for j, char in enumerate(each):
            train_data[i, j, unique_chars.index(char)] = 1
        target_data[i, unique_chars.index(output_char[i])] = 1
    return train_data, target_data, unique_chars, len_unique_chars

In [5]:
def rnn(x, weight, bias, len_unique_chars):
    '''
     uygun bir formata getiriyoruz.
    '''
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, len_unique_chars])
    x = tf.split(x, max_len, 0)
    #graph eşitliklerini yazıyoruz
    cell = tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    prediction = tf.matmul(outputs[-1], weight) + bias
    return prediction


In [6]:
def sample(predicted):
    '''
     hangi harfin döneceğini softmax'a benzer bir şekilde probabilistic hale getiriyoruz.
    '''
    exp_predicted = np.exp(predicted/temperature)
    predicted = exp_predicted / np.sum(exp_predicted)
    probabilities = np.random.multinomial(1, predicted, 1)
    return probabilities


In [7]:
def run(train_data, target_data, unique_chars, len_unique_chars):
    '''
     placeholder'larımızı oluşturuyoruz.
    '''
    x = tf.placeholder("float", [None, max_len, len_unique_chars])
    y = tf.placeholder("float", [None, len_unique_chars])
    #Variable weight'lerini oluşturuyoruz.
    weight = tf.Variable(tf.random_normal([num_units, len_unique_chars]))
    bias = tf.Variable(tf.random_normal([len_unique_chars]))

    prediction = rnn(x, weight, bias, len_unique_chars)
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
    cost = tf.reduce_mean(softmax)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
    #tüm variable'ları initialize ediyoruz.
    init_op = tf.global_variables_initializer()
    #session'ımızı oluşturuyoruz
    sess = tf.Session()
    #variable'ları başlatıyoruz
    sess.run(init_op)
    #yukarıda belirlediğimiz batch size'a göre kaç tane batch olacağını hesaplıyoruz.
    num_batches = int(len(train_data)/batch_size)
    #aşagıdaki for loop kaç epoch kaçıştıracağımızı söylüyor
    for i in range(epoch):
        print("--- Epoch {0}/{1} ---".format(i+1, epoch))
        count = 0
        #aşağıdaki for loop, batch'lerin çalıştırılmasını ifade ediyor.
        for _ in range(num_batches):
            train_batch, target_batch = train_data[count:count+batch_size], target_data[count:count+batch_size]
            count += batch_size
            sess.run([optimizer] ,feed_dict={x:train_batch, y:target_batch})

        #get on of training set as seed
        seed = train_batch[:1:]

        #to print the seed 40 characters
        seed_chars = ''
        for each in seed[0]:
                seed_chars += unique_chars[np.where(each == max(each))[0][0]]
        print("Seed:", seed_chars)

        #trainingimize göre 3000 tane character tahmin ediyoruz.
        for i in range(3000):
            if i > 0:
                remove_fist_char = seed[:,1:,:]
                seed = np.append(remove_fist_char, np.reshape(probabilities, [1, 1, len_unique_chars]), axis=1)
            predicted = sess.run([prediction], feed_dict = {x:seed})
            predicted = np.asarray(predicted[0]).astype('float64')[0]
            probabilities = sample(predicted)
            predicted_chars = unique_chars[np.argmax(probabilities)]
            seed_chars += predicted_chars
        print('Result:', seed_chars)
    sess.close()

In [8]:
%%time
if __name__ == "__main__":
    #get data from https://s3.amazonaws.com/text-datasets/nietzsche.txt
    text = read_data('cb.txt')
    train_data, target_data, unique_chars, len_unique_chars = making_one_hot(text)
    run(train_data, target_data, unique_chars, len_unique_chars)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

----------- Epoch 1/50 -----------
Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimler devlet için bir değerlerine katar ve milletimizin bizim kuruluulu sibabettirimizi bir ana başarımız diyorum. bu bu yanadımız bir kararlı bir delletimizi bu humukun için ensal gelen türkiye’nin için hek katan ile geleceğiz. bu insan daha bir anada bunu başarımız bizim de ortak gerçekleriniz başıran, muhtaramanın kurtlarımız ve yanında başarımız ve hatırla bir olacağız. bir tarafında adımı hatırla bir katarlarını ve hez katan bu yanacaklarımız ve değerli misafir alanın bir arak bizim için daha geceklerini bizim bir dıyorum. bu mustara’daki takım ediyorum.

daha berik sahip çıkar bu vesine ayıklarımız tasımlı sanatlarımız bizim öncelerimiz bir hem de yanında insanlarını bizim için birimizim adımı bu binde sah

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle bu daha büyük geleceğimizden başka bir kezama ve öyle bir başka bir değil, bir başka karşımızda yüzelmesini sahip olmasını devlet veriyoruz. bu teşebbüs ettiğimiz bir defa devlet ortaya konuşturulmasında sanat ve bu daha önümüze bir devlet değildir.

kardeşlerime sanat bu sonunda birilik olan bir kutlamazına geldiler, bu binim inancımız attığımız efedalanın bir kez daha bakanlarımızın itibariyle bu duygularla bir kez daha fazla bu milletimizin boşbakatığımız olmayı hatırlayan bir de tahamet yardıma ve sanat bu ülke inşalı olarak gördük, bu anlamlı büyümesini bir kez daha her edinden geldiler.

son biririmiz geleceğimizi için çalışacağız. çünkü bizim diyorum ki bir başka bir olmuştur. kimse da anlamlı bir başarı bu kararlılığımızın yanında bu inşallah bu anadolu bir şeyler olmasını tek bir yeniden bir şeyine ve tak parti bu tür sinaret başta olmak üzere, son önümüzde bugün boza gayreti içinde bulunan bir de karşımızda yaşamak i

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yüzü bir savaş açık, bu işin balkanlarının ili yıl içinde bulunan karşısında gelen bir karar en kalbi duygularımla, muhabbetle selamlıyorum. bunun için önüne geleceği için güvenliğini başarıya güçlü bir atana insan kuracağız, tankınma hatta bizim karanlık ve yarama alanında olmak üzere, düştüğünde bu karanları teşekkür ediyorum. bunun için bizim milletimize sesleniyorumuz ve bu değerle daha etkiliyorum. biz bir birimizden başarımızda yaşanan insanlarımızın görüyoruz, biz son binde eser almaya başladık, bu toprakları biliyoruz. bakın bir devlet yapıyoruz. bu toprakları karşısında gelecek bir devlet değildir. kimse bu zaman bize göre yüklemek için bakıyorlar, bu tür vatanımız ve insanlık var. bu toprakları bir etane işim gün sonunda akılmış olan bir daha önce muasır medeniyet seviyesinde bir meclis çatışmaların belediye bir milletin evlerine giderik kimsenin bir terör örgütlerine yanlış gelecekleri için haklarını ve milletimize 

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yüz etmeyeceğiz. bu devlet veriyorum. milletimiz hiçbir yanında bir yıkırımı, bu bir yönetim ve yardımcımız olsun diyoruz. bu bir devletmiş olmasını temenni ediyor, bizim için bir yandan sonra bizim için bu duygularla bir kez daha aratına gelen kulaklarıyla yolumuzun ilgili yakında ulaşamayız. asır savan sanayifet başka bir mesarek olarak gördük, bizim böyle bir gazi muktak açıkların arasındaki olarak karşımızdakiler vatandaşlarımın içinde bulunduğu da birkaç aramızda yaşanan müstefak iliyorum.

bu kutlu bir dönem demeye çalışıyoruz. bu vesileyle bu oyunu doğru katkılarına inanıyorum. bu anlayışla bilim hayata geçirmek için aynı şeylerin uluslararası alanda katkı sağlamasından biri değilsen kalımız ve aradaki düşüncelerini bilmeyerek daha özelikle terör eylemlerine baktığımız bir devlettir. bu uygulama insan bakın, her inşahlar ve onları bizim için bu kadar netice kadar bu müstesna de yolumuza, istiklalimizin ve katkılarının s

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yüz yaralarda başında avrupa’nın da bir kez daha işin de bu mücadeleden emir kalkıp bir yere kadar bu da temenni ediyorum.

acaba kurduğumuz ve sınır ötesine karşı çıkartmak için bir terörist, terör örgütlerine sahip çıkmakta başarıyla inanıyorum ki, bu hakka sorumların ve savaş göstermesi de görmek istiyorum. bu ay kararı değerle doğru kurdun bir değerlerde bu işi bilimlerde sağlayan bir an önce hakkını olmak üzere, bilim kuruluşların tarihinin içini de kalmıyoruz. i̇nşallah bu duygularla, tarım teröristinde de hiçbir mücadele yoktur. önümüzdeki öyle savunma sanat içinde daha da karşı geldiler, bu defa örgütler her geçen türkiye’nin bu tür milletimiz var, hakkını da söylediğim bizim için bu sonuçları terör örgütlerine verdiğimiz bu gençlerimizi de bir anlayışta çok daha önce milletimiz bize değil mirasına bakmak, bizim gerektiğinde aynı harekete gelmesine inanıyorum.

bu mücadelemizi hatırlayacağımız bu anlamlı genç başbakanl

...

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yad ediyorum. çünkü bizim kitabımızda karanlık ve milletimizin birinci barış ve irak’taki son bir kez daha inanıyorum. bize güç içinde olduklarının bu doğrultuda aynı anda bir devleti için de şöyle bir an önce hayata geçirmek istiyorum. bu görev yapan üstünden içikilir yaptığımız bu durum bu milletin geleceğine verecektir.

en sahid vermiştir.

bugün sizi gördüm, tarihin bir devlet anlayışı yok rekayür elde edilere bizi bir ne diyorsunuz? tem bakın olarak görüyoruz. bu aramızda birim ve mağdurlarımızı aynı anda bir devlettir.

bugün de topraklarımıza bir kez daha teşekkür ediyorum.

türkiye’nin bu kardeşlerimizin tabii karşından doğuşarsk. kurallarının hakkıyla doktoyduki karşılamış, bunun için söyleyin, bundan sonra da bu duygularla bir kez daha da muhafakaa ekonomik terörist olarak ilan da bir doğrultudur.

türkiye’nin bir yanında da bir oldukların tekabü ediyorları bizim için dünyaya kadar her alanda rüzgâsını sistemiyle ba

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yâd ediyorum. sizlere sevgiyle, saygıyla selamlıyorum. i̇stiklal harbi fariş atarakların karanlık ve tevkiler alanında bir başka teröristlerin kuruluşumuzun insanı yakından çok önemli olduklarını bildiğimiz varsmle vatandaşımızdan biri olarak gördüğümüz için benel sahabi olan giziyiz ve marazların atatürk’ün bilim ve insanlar muhabbetle seltmlarımızı saygıyla prolarlık ve kulağımıza bakmak zorunda bu bir anlama gönülden daima müslüman değildir, bu geniştir bizim için her fırsatta kalmamıştır.

sizleri en kalbi duygularımla, hasretle, muhabbetle selamlıyorum.

doğu alanda bizim bir devlet onlara büyük bir bu yılın bu saldırıların inşa etmesini isteyonlar. bu hassasiyeti kuracağınız dilen hazırlıkların, türkiye’nin bu kadar seviciye olan irak’taki sürece araç olacaktır. zira bu değerlerimizi dinledik, biliyorsunuz sizlere sevgilerimi, saygılarımı sunuyorum. türkiye olarak bizim için hiçbir ilken, demokratik üzerin bir kez daha b

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yad ediyorum.

değerli misafirler;

bu sürecinde gerçek yüzünün gönül üzerine gittiğimiz bu sebeple bir masal tarihi hatırlayı besleyen bir bakım olarak görmek memur veyla bu kusum daha verimli bir hatiate varsa, tam terör örgütlerini tamamek yok değildir.

bugün terörün, tüm bu öğrencilerimizi birlikte görüşmeler olmuştur. bu konuda yatırım biliyoruz.

arak, parının daha büyük bir son da makamına doğru yallaştirmesini için terör örgütlerinin içinden sonra da bu anlayışla hareket ediyorlar ki bunun için bize yatırım böyle bir düşüncelerinin bu çalışmalarını yakın artık boyunca da değil geride bırakmıya çalışıyoruz.

aotan geniş bir dönemden biri haline getirmeye devam edeceğiz. bir kez daha sahip çıktı. bunun için tabii ki bu dre fatikatının başka bir kez daha son olarak görmeye başladh. i̇nsan etabını ders kitaplarına rağmen suriye’deki kültürümüzdan, siz insan bastıranlar dünyanın desteği sürecini yaşadıklarımız bu nesilo yö

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle yad ediyorum. bunun için bir an önce geliştirmeliyiz. yani gelip beçlerdi duyarı kiiliyore yollarını bölgede yaşadıkları bir yönde bir araya gelmiş bulunanlar içinde her birinize ayrı ayrılşımdan diliyorum, i̇şte son dünyaya baktığımız gibi, bu da istemesini sağlayacak bir kez daha çünkü bir kez daha bir kez daha saygıdeğer hoş bağı kültürel ve bu kararlı bir alanda bir katkı tabin eden her şeyi vardır, olmayacak, nasıl bu sistemine sahip çıkmaktır.

ne olduğunu dönmüş zamanlar muhteşel başkanlığını tarihinin uzunluğuna inanıyorum. en az bir terari görmüyoruz. bu vatan için hayırlara yaslarda bulunan bu sorunun mevcut yardım düşürmeleri ise başladığı da almayanlar allah’ın indirememezli bir hizmetlere almadan müslümanların, tüm dünyanın da bu milletin her sınırlarımızın önüne geçireceğiz.

bugün de türkiye, ama bu anlamlı olacaktır. ama siz kişileöne göre bize karşı selamın asil bir araya değildir. bu anlamlı bir dönüm noktası

Seed: z daha rahmetle, taz
Result: z daha rahmetle, tazimle ve ilk cina, tüm ümk demorslü o kandileri gençleri olarak gördüğümüz adına teşekkür ediyorum. sizlerle birlikte bu kadar tek vatan, altındadık.

ak parti, bölgemizde yaşanan kurulmaya çalışıyoruz. bu değerlerimize almıştıra uygun işte bu anlamlı olarak kullanılamayacağız.

biz batırım olarak bu teşebbüs kalıte’nin başta olmak üzere, tesliminet değil kuraklımının bu coğrafina inşaımız olmadığımız bu devleti ve ilgili kim olmak üzere, bizim için gelen kendi topraklarını avrupa’da birik ama bu konuda türkiye’de başbakan değerleri en kalbi har karikatların pek çok terör eylemleri hangi inanıyla işler demekten olduğu artık bu süreci başlamasını asla gerekliden gelmesi olarak devam etmemiz gerekiyor. sadece bu kutlu bir şey varsa orunla görüşmeler kurulmasına rağmen bu değerlerimize sahip çıkma yürütülerken kendi alanında kendi son hesap veriyoruz, hep birlikte karşımızdaki anlamına dahi ne kadar çalışma ortamı gelecek bir taraftan 